In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from google.cloud import bigquery

In [3]:
def get_sp500_tickers():
    return pd.read_csv("tickers.csv", header=None)[0].tolist()


# Download previous day's data
def download_previous_day_data(tickers):
    """
    Downloads daily data for the tickers from the previous day.
    """
    # start_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    # end_date = (datetime.now() - timedelta(days=0)).strftime("%Y-%m-%d")

    current_date = datetime.now() - timedelta(days=1)
    while not pd.Timestamp(current_date).isoweekday() in range(
        1, 6
    ):  # Monday (1) to Friday (5)
        current_date -= timedelta(days=1)

    start_date = current_date.strftime("%Y-%m-%d")
    end_date = (current_date + timedelta(days=1)).strftime("%Y-%m-%d")

    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date,
        end=end_date,
        interval="1d",
        group_by="ticker",
        threads=True,
    )
    if data.empty:
        print(f"No data available for {start_date}.")
        return pd.DataFrame()  # Returns an empty DataFrame if no data is available
    # Restructure the data
    data = data.stack(level=0, future_stack=True).reset_index()
    data.columns = [
        "Date",
        "Ticker",
        "Open",
        "High",
        "Low",
        "Close",
        "Adj Close",
        "Volume",
    ]
    return data


# Download data for all tickers
def download_sp500_data(tickers, start_date="2000-01-01", end_date=None):
    """
    Downloads daily data for all S&P 500 tickers.
    """
    # Fetch the data via yfinance
    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date,
        end=end_date,
        interval="1d",
        group_by="ticker",  # Organizes the data by ticker
        threads=True,  # Parallel download
    )

    data = data.stack(level=0, future_stack=True).reset_index()  # Make tickers a column
    data.columns = [
        "Date",
        "Ticker",
        "Open",
        "High",
        "Low",
        "Close",
        "Adj Close",
        "Volume",
    ]

    return data


# Load data into BigQuery
def load_data_to_bigquery(df, table_id, project_id):
    """
    Loads a DataFrame into a BigQuery table.

    Arguments:
        df : pandas.DataFrame containing the data to load.
        table_id : Full ID of the BigQuery table (e.g., dataset.table_name).
        project_id : Google Cloud project ID.
    """
    client = bigquery.Client(project=project_id)

    # Convert the data to BigQuery format
    job = client.load_table_from_dataframe(df, table_id)

    # Wait for the job to complete
    job.result()
    print(f"Data has been loaded into {table_id}")


# Load data into a temporary table
def load_to_temp_table(client, df, temp_table_id):
    """
    Loads data into a temporary BigQuery table.
    """
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  # Overwrites existing data
    )
    job = client.load_table_from_dataframe(df, temp_table_id, job_config=job_config)
    job.result()
    print(f"Data loaded into temporary table {temp_table_id}")


# Merge data with the main table
def merge_into_main_table(client, temp_table_id, main_table_id):
    """
    Merges data from the temporary table into the main table.
    """
    query = f"""
    MERGE `{main_table_id}` AS main
    USING `{temp_table_id}` AS temp
    ON main.Date = temp.Date AND main.Ticker = temp.Ticker
    WHEN NOT MATCHED THEN
      INSERT (Date, Ticker, Open, High, Low, Close, `Adj Close`, Volume)  -- `Adj Close` with a space
      VALUES (temp.Date, temp.Ticker, temp.Open, temp.High, temp.Low, temp.Close, temp.`Adj Close`, temp.Volume)
    """
    job = client.query(query)
    job.result()
    print(f"Data merged into main table {main_table_id}")


def fill_table():
    # Step 1: Fetch tickers
    print("Downloading S&P 500 tickers...")
    sp500_tickers = get_sp500_tickers()
    print(f"Number of tickers fetched: {len(sp500_tickers)}")

    # Step 2: Download data
    print("Downloading daily data...")
    sp500_data = download_sp500_data(sp500_tickers, start_date="2000-01-01")

    # # Step 3: Load into BigQuery
    # print("Loading data into BigQuery...")
    # PROJECT_ID = "quant-dev-442615"  # Replace with your project ID
    # DATASET_ID = "financial_data"  # Replace with your dataset name
    # TABLE_ID = "sp500_data"  # Table name

    # # Load data
    # load_data_to_bigquery(
    #     sp500_data,
    #     table_id=f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}",
    #     project_id=PROJECT_ID,
    # )
    return sp500_data


def add_daily():
    # Step 1: Configuration
    PROJECT_ID = "quant-dev-442615"
    DATASET_ID = "financial_data"
    MAIN_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.sp500_data"
    TEMP_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.temp_sp500_data"

    # Fetch tickers
    print("Downloading S&P 500 tickers...")
    tickers = get_sp500_tickers()

    # Step 2: Download data
    print("Downloading data for the previous day...")
    sp500_data = download_previous_day_data(tickers)
    if sp500_data.empty:
        print("No data available. Execution stopped.")
        return  # Stop execution if no data is available
    sp500_data["Date"] = pd.to_datetime(sp500_data["Date"]).dt.date
    # Convert the Volume column to integer
    sp500_data["Volume"] = sp500_data["Volume"].fillna(0).astype(int)

    # Step 3: Load into BigQuery
    client = bigquery.Client(project=PROJECT_ID)
    print("Loading data into a temporary table...")
    load_to_temp_table(client, sp500_data, TEMP_TABLE_ID)

    # Step 4: Merge with the main table
    print("Merging data with the main table...")
    merge_into_main_table(client, TEMP_TABLE_ID, MAIN_TABLE_ID)

In [5]:
tickers = get_sp500_tickers()

In [7]:
sp500_data = download_previous_day_data(tickers)

[*********************100%***********************]  517 of 519 completed

519 Failed downloads:
['LH', 'JBL', 'EMN', 'TFC', 'FMC', 'NSC', 'TRV', 'LMT', 'FI', 'SPY', 'GLD', 'ARE', 'MCO', 'NVR', 'JCI', 'CVS', 'BLDR', 'MS', 'JPM', 'DVA', 'TECH', 'FICO', 'TRMB', 'IQV', 'AMGN', 'MSFT', 'WYNN', 'IP', 'MOH', 'PFG', 'IBM', 'JBHT', 'NCLH', 'KMB', 'ZBRA', 'HAL', 'PGR', 'JNPR', 'DOC', 'CME', 'WDC', 'FFIV', 'DAY', 'NWS', 'HSY', 'MCD', 'ROP', 'EQIX', 'MTB', 'ADSK', 'UAL', 'CRL', 'NXPI', 'F', 'BSX', 'WST', 'ANET', 'BAX', 'TROW', 'CF', 'MET', 'VRSN', 'CCI', 'SPG', 'KKR', 'SYK', 'RF', 'EA', 'HUM', 'OTIS', 'ICE', 'FOX', 'MMM', 'CRM', 'KEYS', 'AIG', 'AEP', 'EQT', 'KLAC', 'CNP', 'PNC', 'HII', 'DFS', 'LRCX', 'PG', 'JKHY', 'URTH', 'DECK', 'XEL', 'IVZ', 'ZBH', 'DGX', 'CB', 'BTC-USD', 'ROL', 'KIM', 'HCA', 'STE', 'EXPD', 'SYY', 'FAST', 'MCHP', 'CL=F', 'DD', 'HG=F', 'NTAP', 'MHK', 'JNJ', 'GD', 'LOW', 'APTV', 'EQR', 'HSIC', 'NEE', 'AFL', 'OMC', 'CBRE', 'MSI', 'LEN', 'POOL', 'ETN', 'ERIE', 'IDXX', 'CARR', 'MMC',

No data available for 2025-08-08.


In [ ]:
start_date = "2000-01-01"
end_date = None
data = yf.download(
    tickers=" ".join(tickers),
    start=start_date,
    end=end_date,
    interval="1d",
    group_by="ticker",  # Organizes the data by ticker
    threads=True,  # Parallel download
)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3690624467.py, line 1)

In [ ]:
data = data.stack(level=0, future_stack=True).reset_index()  # Make tickers a column
data.columns = [
    "Date",
    "Ticker",
    "Open",
    "High",
    "Low",
    "Close",
    "Adj Close",
    "Volume",
]

""


In [3]:
# Fetch the data via yfinance
start_date = "2000-01-01"
end_date = None
tickers = get_sp500_tickers()
data = yf.download(
    tickers=" ".join(tickers),
    start=start_date,
    end=end_date,
    interval="1d",
    group_by="ticker",  # Organizes the data by ticker
    threads=True,  # Parallel download
)

[*********************100%***********************]  519 of 519 completed

1 Failed download:
['TSCO']: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


In [ ]:
df_work = data.stack(level=0, future_stack=True).reset_index()